# Deep Learning with PyTorch: A 60 Minute Blitz

## Autograd: automatic differentiation

Central to all neural networks in PyTorch is the `autograd` package. Let's briefly visit this, and we will then go to train our first neural network.

The `autograd` package provides automatic differentiation for all operations on tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

### Variable

The class `autograd.Variable` is the central class of the package. It wraps a tensor, and supports nearly all of the operations defined on it. Once you finish your computation you can call `.backward()` and have all the gradients computed automatically.

You can access the raw tensor through the `.data` attribute, while the gradient with respect to this variable is accumulated into `.grad`.

There is one more class which is very important for autograd implementation - a `Function`. `Variable` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each variable has a `.grad_fn` attribute that references a `Function` that has created the `Variable` (except for variables created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a `Variable`. If `Variable` is a scalar (i.e. it holds a one-element data), you don't need to specify any arguments to `backward()`, however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

In [1]:
import torch
from torch.autograd import Variable

Create a variable:

In [2]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



Do an operation on a variable:

In [3]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



Variable `y` was created as a result of an operation, so it has a `grad_fn`.

In [4]:
print(y.grad_fn)

Do some more operations on `y`:

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



### Gradients

Let's now backpropagate: `out.backward()` is equivalent to doing `out.backward(torch.Tensor([1.0]))`

In [6]:
out.backward()

# Print gradients d(out)/dx
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



What happened here? Well, $out = \frac{1}{4} \sum_i z-i$ and $z_i = 3(x_i + 2)^2$, so $z_i \rvert_{x_i = 1}$ = 27 and therefore

$$\frac{\partial(out)}{\partial x} = \frac{3}{2}(x_i + 2)$$

hence

$$\frac{\partial(out)}{\partial x} \rvert_{x_i = 1} = \frac{9}{2} = 4.5$$

You can do many crazy things with autograd!

In [7]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print(y)

Variable containing:
 -123.4616
-1517.6437
  667.5115
[torch.FloatTensor of size 3]



In [8]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



Documentation of `Variable` and `Function`